
### Model: Pre-trained Inception V3 - SpatialTransformer

#### This notebooks trains the Localization Network which outputs an affine transformation matrix. It uses the pre-trained weights of the Inception V3 trained on the training data fra Aarhus University. The final layer of the Inception V3 is then further trained using the output of the ST-network

#### Load the weights of the trained Inception Architecture and train the SpatialTransformer. Update the weights of the final layer in the Inception V3. 

The resulting weights are saved, and are useable in the Evaluate notebook. The images will not be uploaded, and so this notebook is not runable. When training, images has been packaged in a .pickle file. The weights are saved in the Keras .HDF5 format. 

Authors:
* s134859 Nicolai Mogensen
* s134569 Tobias Slot Jensen
* s144242 David Frich Hansen

References:
* SpatialTransformer Keras Implementation Hello2all: https://github.com/hello2all/GTSRB_Keras_STN

In [ ]:
DIM = 299
import numpy as np
from label_image import *
import glob
import os
from pathlib import Path
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
#from PIL import Image,ImageOps
from skimage.io import imread
from skimage.transform import resize
from keras.layers import (Activation, Dense, Dropout, Flatten,
                          Lambda, MaxPooling2D)
import performance_measures
from keras.layers.convolutional import Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, model_from_json, Model
from keras.regularizers import l2
from keras.utils import np_utils
from spatial_transformer import SpatialTransformer
from keras.applications import InceptionV3
import pickle

from keras.callbacks import ModelCheckpoint,LambdaCallback,Callback
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import MinMaxScaler
from scipy.special import expit
import keras.backend as K

In [ ]:
#Load the pickled data, this data is NOT uploaded as it is confidential
with open('images.pickle', 'rb') as handle:
    data = pickle.load(handle)


In [ ]:
# Reference "Hello2All"
def locnet():
    num_weights = 6
    b = np.zeros((2, 3), dtype='float32')
    
    # These parameters indicate zoom
    # Initialize these to be less than one to avoid infinite zoom-out
    b[0, 0] = 0.40
    b[1, 1] = 0.40
    
    W = np.repeat(np.array([0.0,0.0,0.0,0.0,0.0,0.0],dtype='float32'),num_weights).reshape(num_weights,6,order="F")
    weights = [W, b.flatten()]
    
    locnet = Sequential()

    locnet.add(Conv2D(16, (7, 7), padding='valid', input_shape=(299, 299, 3)))
    locnet.add(MaxPooling2D(pool_size=(2, 2)))
    locnet.add(Conv2D(32, (5, 5), padding='valid'))
    locnet.add(MaxPooling2D(pool_size=(2, 2)))
    locnet.add(Conv2D(64, (3, 3), padding='valid'))
    locnet.add(MaxPooling2D(pool_size=(2, 2)))

    locnet.add(Flatten())
    locnet.add(Dense(128))
    locnet.add(Activation('relu'))
    locnet.add(Dense(64))
    locnet.add(Activation('relu'))
    locnet.add(Dense(num_weights))
    locnet.add(Activation('relu'))
    locnet.add(Dense(6, weights=weights))

    return locnet

def conv_model(input_shape=(299, 299,3)):
    
    # Part that contains ST-net
    model = Sequential()
    model.add(Lambda(
        lambda x: x,
        input_shape=input_shape,
        output_shape=input_shape))
    
    # Batch normalize to "relax" the weights and avoid explosion
    model.add(BatchNormalization())
    stnet = SpatialTransformer(localization_net=locnet(),
                                 output_size=(299,299)) #Resize to 299x299
    
    model.add(stnet)
    
    # Inception, load weights from HDF5 file, previously trained
    model_2 = Sequential()
    inc_v3 = InceptionV3(include_top=False, weights='imagenet', pooling='max',classes=2)
 
    # Don't train the layers, as everything is already trained
    for layer in inc_v3.layers:
        layer.trainable = False
    
    model_2.add(inc_v3)
    #model_2.add(BatchNormalization())
    output = Dense(2, activation='softmax')
    # Lets keep training the final layer, to obtain end-to-end
    output.trainable = True
    
    
    model_2.add(output)
    # Load the inception weights as obtained in another notebook
    model_2.load_weights("inception_pure.hdf5")
   
    # Add the Inception Conv Net after the ST-net
    model.add(model_2)
    
    return model

In [ ]:
batch_size = 64
epochs = 30 # Previously this was 300 epochs, but the weights explodes after too many epochs
model = conv_model()

model.summary()
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=adam,metrics=['accuracy'])

In [ ]:
# Override Keras callback to plot transformed images after each epoch
class PlotTestset(Callback):
    # on_epoch_end is a keras function which we overwrite
    def on_epoch_end(self, batch, logs={}):
        y_prob = model.predict(data["X_test"]) # Probabilities from Softmax
        st_img = intermediate_layer_model.predict(data["X_test"]) # Transformed images
        y_pred = np.argmax(y_prob,axis=1) # Classification
        
        # subplot parameters
        p1 = 1
        p2 = 2
        # Plot just the first 3 images
        for i in range(3):
            plt.figure(figsize = (12,12))
            plt.subplot(10,2,p1)
            # Normalize the images so we can plot them
            tmp_img = st_img[i]+np.abs(np.amin(st_img[i]))
            tmp_img = tmp_img/(np.amax(st_img[i])-np.amin(st_img[i]))
            print("Shape of output:",st_img[i].shape)
            print("Shape of input:",data["X_test"].shape)
            plt.imshow(tmp_img)
            #plt.imshow(st_img[i])
            
            plt.axis('off')
            plt.subplot(10,2,p2)
            plt.imshow(data["X_test"][i])
            plt.axis('off')
            
            acne_pred = "Vulgaris"
            acne_true = "Vulgaris"
            if(y_pred[i] == 1):
                acne_pred = "Rosacea"
            if(data["y_test"][i] == 1):
                acne_true = "Rosacea"
                
            print("\nPrediction: " + str(y_pred[i]) + " " + acne_pred)
            print("True: " + str(data["y_test"][i]) + " " + acne_true)
            plt.show()
            p1 += 2
            p2 += 2

In [ ]:
# Keras does validation splits for us, so combine the Training and Validation data, and let Keras split it. 
data["X_train_c"] = np.concatenate((data["X_train"],data["X_valid"]),axis=0)
data["y_train_c"] = np.concatenate((data["y_train"],data["y_valid"]),axis=0)

# Intermediate model that lets us take out the transformed image to plot.
intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer("spatial_transformer_1").output)
# checkpointer lets us save the weights at each epoch
checkpointer = ModelCheckpoint(filepath="STnet.hdf5", verbose=1, save_best_only=False, save_weights_only=True)
# plot callback
plottest = PlotTestset()

# Train the model with the parameters
try:
    model.fit(data["X_train_c"], data["y_train_c"],
                batch_size=batch_size,
                epochs=epochs,
                validation_split=0.1,
                shuffle=True,
                callbacks=[plottest,checkpointer])
                
except KeyboardInterrupt:
    print("training interrupted")


In [ ]:
# Evaluate model on test set
score = model.evaluate(x=data["X_test"], y=data["y_test"])
print(score)

In [ ]:
from performance_measures import *

In [ ]:
y_prob = model.predict(data["X_test"])
y_pred = np.argmax(y_prob,axis=1)
y_true = data["y_test"]

AUC = get_roc_auc(y_true,y_prob[:,1])
accuracy = score[1]

#True Negative Rate 0,0
specificity = get_specificity(y_true,y_pred)
#True Positive Rate 1,1
sensitivity = get_sensitivity(y_true,y_pred)

print("Accuracy:",accuracy)
print("Specificity:",specificity)
print("Sensitivity:",sensitivity)

### Print some images from the test set (Again, we can not show this)

In [ ]:
y_prob = model.predict(data["X_test"]) # Probabilities from Softmax
st_img = intermediate_layer_model.predict(data["X_test"]) # Transformed images
y_pred = np.argmax(y_prob,axis=1) # Classification

# subplot parameters
p1 = 1
p2 = 2
# Plot just the first 3 images
for i in range(3):
    plt.figure(figsize = (12,12))
    #plt.subplot(10,2,p1)
    # Normalize the images so we can plot them
    tmp_img = st_img[i]+np.abs(np.amin(st_img[i]))
    tmp_img = tmp_img/(np.amax(st_img[i])-np.amin(st_img[i]))
    #print("Shape of output:",tmp_img[i].shape)
    #print("Shape of st_img:",st_img[i])
    #print("Shape of input:",data["X_test"].shape)
    
    plt.imshow(tmp_img)
    #plt.imshow(st_img[i])

    plt.axis('off')
    #plt.subplot(10,2,p2)
    #plt.imshow(data["X_test"][i])
    #plt.axis('off')

    acne_pred = "Vulgaris"
    acne_true = "Vulgaris"
    if(y_pred[i] == 1):
        acne_pred = "Rosacea"
    if(data["y_test"][i] == 1):
        acne_true = "Rosacea"

    print("\nPrediction: " + str(y_pred[i]) + " " + acne_pred)
    print("True: " + str(data["y_test"][i]) + " " + acne_true)
    plt.show()
    p1 += 2
    p2 += 2